# 🧬 Olink Reveal — Quality Control Workflow

This notebook provides a complete, step-by-step QC pipeline for **Olink® Reveal** NGS-based proteomics datasets using the [`OlinkAnalyze`](https://cran.r-project.org/package=OlinkAnalyze) R package.

---

## Workflow Overview

| Step | Description |
|------|-------------|
| 1 | Install & load libraries |
| 2 | Import NPX data |
| 3 | Integrate Limit of Detection (LOD) |
| 4 | Review Olink QC flags |
| 5 | Prepare clean data for visualisation |
| 6 | QC scatter plot (IQR vs Median) |
| 7 | PCA plot — global sample overview |
| 8 | NPX distribution plot |
| 9 | Identify & remove outliers |
| 10 | Missing frequency / below-LOD summary |
| 11 | Plate/batch effect check |
| 12 | Export QC-passed data |

---

> **Note:** All file paths marked with `"path/to/..."` must be replaced with the actual paths to your data files before running.

---
## Step 1 — Install & Load Libraries

The primary package is **`OlinkAnalyze`**, available on CRAN. Supporting packages (`dplyr`, `ggplot2`, `stringr`, `ggpubr`) are used for data manipulation and visualisation.

Uncomment the `install.packages()` lines if running for the first time.

In [ ]:
# Uncomment to install on first run
# install.packages("OlinkAnalyze")
# install.packages(c("dplyr", "ggplot2", "stringr", "ggpubr"))

library(OlinkAnalyze)
library(dplyr)
library(ggplot2)
library(stringr)
library(ggpubr)

message("✅ Libraries loaded successfully")

---
## Step 2 — Import NPX Data

`read_NPX()` imports Olink NPX output files (`.parquet`, `.xlsx`, or `.csv`) and converts them into a **long-format tibble**. No prior modifications to the exported file should be made.

### Key columns in the imported data

| Column | Description |
|--------|-------------|
| `SampleID` | Sample name or ID |
| `SampleType` | SAMPLE, NEGATIVE_CONTROL, etc. |
| `OlinkID` | Unique assay identifier assigned by Olink |
| `Assay` | Common gene name |
| `NPX` | Normalised Protein eXpression (log₂ scale) |
| `PCNormalizedNPX` | Plate-control normalised NPX |
| `Normalization` | `"Plate control"` or `"Intensity"` |
| `QC_Warning` / `SampleQC` | Olink QC pass/fail flag |
| `PlateID` | Plate identifier |
| `MissingFreq` | Fraction of samples below LOD for each assay |
| `Panel` | Olink panel name |

In [ ]:
# ── Option A: Single NPX file ─────────────────────────────────────────────────
npx_data <- read_NPX("path/to/your/Reveal_NPX_file.parquet")

# ── Option B: Multiple NPX files from the same directory ─────────────────────
# npx_data <- list.files(
#   path       = "path/to/dir/with/NPX/files",
#   pattern    = "parquet$",        # or "csv$" or "parquet$|csv$"
#   full.names = TRUE
# ) |>
#   lapply(OlinkAnalyze::read_NPX) |>
#   dplyr::bind_rows()

# Inspect data structure
cat("Dimensions:", nrow(npx_data), "rows x", ncol(npx_data), "columns\n")
glimpse(npx_data)

In [ ]:
# Summary of SampleType breakdown
npx_data |>
  count(SampleType, name = "n_rows") |>
  print()

# Summary of normalization type used
npx_data |>
  distinct(SampleID, Normalization) |>
  count(Normalization)

---
## Step 3 — Integrate Limit of Detection (LOD)

Olink Reveal is an **NGS-based product**, so LOD is **not** pre-embedded in the exported file and must be calculated separately using `olink_lod()`.

### Choosing the right LOD method

| Condition | Recommended method | Notes |
|-----------|-------------------|-------|
| ≥ 10 passing negative controls | `"NCLOD"` | Project-specific; preferred |
| < 10 passing negative controls | `"FixedLOD"` | Lot-specific; download CSV from [olink.com](https://olink.com/knowledge/documents) |
| Want to compare both | `"Both"` | Adds 4 LOD columns; rename preferred one to `LOD` |

### LOD columns added by `olink_lod()`

| Column | Content |
|--------|---------|
| `LOD` | For PC-normalized: same as `PCNormalizedLOD`. For intensity-normalized: intensity-adjusted LOD |
| `PCNormalizedLOD` | Plate-control normalised LOD (unadjusted) |

> **NC LOD method:** LOD = median PC-normalised NPX across NCs + 3 SD (or + 0.2 NPX, whichever is larger). For assays with < 150 counts in all NCs, count-based LOD is converted to NPX.

In [ ]:
# Count passing negative controls
nc_count <- npx_data |>
  filter(SampleType == "NEGATIVE_CONTROL") |>
  distinct(SampleID) |>
  nrow()

cat("Number of negative controls detected:", nc_count, "\n")
cat("Recommended LOD method:", ifelse(nc_count >= 10, "NCLOD (project-specific)", "FixedLOD (download CSV from olink.com)"), "\n")

In [ ]:
# ── Option A: NC-based LOD (>= 10 passing NCs) ────────────────────────────────
npx_data <- olink_lod(npx_data, lod_method = "NCLOD")

# ── Option B: Fixed LOD (< 10 NCs) ────────────────────────────────────────────
# fixedLOD_filepath <- "path/to/Reveal_fixedLOD.csv"   # download from olink.com
# npx_data <- olink_lod(npx_data,
#                       lod_file_path = fixedLOD_filepath,
#                       lod_method    = "FixedLOD")

# ── Option C: Both methods ─────────────────────────────────────────────────────
# npx_data <- olink_lod(npx_data,
#                       lod_file_path = fixedLOD_filepath,
#                       lod_method    = "Both")
# # Then rename your preferred LOD column:
# npx_data <- npx_data |> rename(LOD = NCLOD)

# Verify LOD columns were added
cat("LOD columns present:", paste(intersect(c("LOD", "PCNormalizedLOD", "NCLOD", "FixedLOD"), colnames(npx_data)), collapse = ", "), "\n")

# Preview LOD values for a single assay
npx_data |>
  filter(SampleType == "SAMPLE") |>
  select(SampleID, OlinkID, Assay, NPX, PCNormalizedNPX, LOD, PCNormalizedLOD, Normalization) |>
  head(5)

---
## Step 4 — Review Olink QC Flags

Olink's own software assigns QC flags to each sample based on internal controls. These are stored in the `QC_Warning` or `SampleQC` column (column name varies by product version).

- **Pass** — sample passed all Olink QC criteria
- **Warning** — sample may have quality issues; review before including
- **Fail** — sample failed QC; should generally be excluded

More details: [Olink FAQ — Quality Control](https://olink.com/knowledge/faq?query=quality%20control)

In [ ]:
# Auto-detect the QC column name
qc_col <- if ("SampleQC" %in% colnames(npx_data)) "SampleQC" else "QC_Warning"
cat("QC column detected:", qc_col, "\n")

# Summarise QC status across all samples
qc_summary <- npx_data |>
  filter(SampleType == "SAMPLE") |>
  distinct(SampleID, PlateID, .data[[qc_col]]) |>
  count(.data[[qc_col]], name = "n_samples")

print(qc_summary)

In [ ]:
# List all samples with a QC Warning or Fail
qc_flagged <- npx_data |>
  filter(SampleType == "SAMPLE",
         !.data[[qc_col]] %in% c("Pass", "PASS")) |>
  distinct(SampleID, PlateID, .data[[qc_col]])

cat("Samples with QC Warning/Fail:", nrow(qc_flagged), "\n")
print(qc_flagged)

---
## Step 5 — Prepare Clean Data for Visualisation

Before running QC visualisation functions, we filter out:
- Internal control samples (negative controls, plate controls, etc.)
- Control assays (e.g. housekeeping/control probes)

This ensures the QC plots reflect only the biological study samples.

In [ ]:
npx_samples <- npx_data |>
  # Keep only biological samples
  filter(SampleType == "SAMPLE") |>
  # Remove samples with IDs suggesting they are controls
  filter(!str_detect(SampleID, regex("control|ctrl|blank", ignore_case = TRUE))) |>
  # Remove control assays
  filter(!str_detect(Assay,    regex("control|ctrl",       ignore_case = TRUE)))

cat("Samples retained for QC visualisation:",
    n_distinct(npx_samples$SampleID), "samples\n")
cat("Assays retained:",
    n_distinct(npx_samples$OlinkID), "\n")

---
## Step 6 — QC Scatter Plot: IQR vs. Median

`olink_qc_plot()` generates a scatter plot of **IQR vs. median NPX** for each sample, faceted by panel.

- **Horizontal dashed lines** = ± 3 SD from the mean IQR → samples outside are IQR outliers
- **Vertical dashed lines** = ± 3 SD from the mean sample median → samples outside are median outliers
- Colouring by `PlateID` helps reveal **plate-level batch effects**

This is the **recommended first QC check** before any downstream analysis.

In [ ]:
# Basic QC plot (no colour grouping)
qc_plot <- olink_qc_plot(npx_samples)

# Display the first panel's plot
qc_plot[[1]]

In [ ]:
# QC plot coloured by PlateID — useful to detect plate-level batch effects
qc_plot_plate <- olink_qc_plot(npx_samples, color_g = "PlateID")
qc_plot_plate[[1]]

In [ ]:
# Save all panel QC plots to disk
for (i in seq_along(qc_plot)) {
  ggsave(
    filename = paste0("QC_IQR_median_panel_", i, ".png"),
    plot     = qc_plot[[i]],
    width = 10, height = 6, dpi = 300
  )
}
message("QC plots saved.")

---
## Step 7 — PCA Plot: Global Sample Overview

Principal Component Analysis (PCA) reduces the high-dimensional NPX data to the axes of greatest variation. `olink_pca_plot()` is used to:

- **Identify individual outlier samples** — samples far from the main cluster
- **Identify group-level effects** — samples clustering by a biological variable
- **Detect batch effects** — samples clustering by plate or run

The `label_samples = TRUE` option labels each point with its SampleID, making it easy to identify outliers by name.

In [ ]:
# PCA with sample labels — helps identify individual outlier samples
pca_labeled <- olink_pca_plot(npx_samples, label_samples = TRUE, quiet = TRUE)
pca_labeled[[1]]

In [ ]:
# PCA coloured by PlateID — reveals potential batch effects
pca_plate <- olink_pca_plot(npx_samples, color_g = "PlateID", quiet = TRUE)
pca_plate[[1]]

In [ ]:
# PCA coloured by a clinical/biological variable (replace "Group" with your column)
# pca_group <- olink_pca_plot(npx_samples, color_g = "Group", quiet = TRUE)
# pca_group[[1]]

# Save PCA plots
ggsave("PCA_labeled.png",  plot = pca_labeled[[1]], width = 9, height = 7, dpi = 300)
ggsave("PCA_by_plate.png", plot = pca_plate[[1]],   width = 9, height = 7, dpi = 300)
message("PCA plots saved.")

---
## Step 8 — NPX Distribution Plot

`olink_dist_plot()` plots the NPX density distribution for each sample, faceted by panel. 

Use this to spot samples whose overall NPX distribution is **shifted** relative to the rest of the cohort — a sign of technical issues such as sample degradation, insufficient input, or pipetting errors.

In [ ]:
# Distribution plot coloured by PlateID
dist_plot <- olink_dist_plot(npx_samples, color_g = "PlateID")
dist_plot[[1]]

In [ ]:
ggsave("NPX_distribution.png", plot = dist_plot[[1]], width = 12, height = 6, dpi = 300)
message("Distribution plot saved.")

---
## Step 9 — Identify & Remove Outliers

Samples are flagged as outliers if they meet **any** of the following criteria:

1. **Olink QC flag** — `QC_Warning` / `SampleQC` is not `"Pass"`
2. **Median outlier** — sample median NPX is > 3 SD from the panel mean sample median
3. **IQR outlier** — sample IQR is > 3 SD from the panel mean IQR

> ⚠️ **Important:** Always investigate *why* a sample is flagged before excluding it. A sample may be a statistical outlier due to biology (e.g. a patient with extreme disease severity) rather than a technical artefact. In such cases, the sample may be retained with additional normalisation.

In [ ]:
# Compute per-sample median and IQR, then flag outliers vs. panel ±3 SD bounds
qc_stats <- npx_samples |>
  group_by(SampleID, Panel) |>
  summarise(
    sample_median = median(NPX, na.rm = TRUE),
    sample_IQR    = IQR(NPX,    na.rm = TRUE),
    .groups       = "drop"
  ) |>
  group_by(Panel) |>
  mutate(
    mean_median    = mean(sample_median),
    sd_median      = sd(sample_median),
    mean_IQR       = mean(sample_IQR),
    sd_IQR         = sd(sample_IQR),
    outlier_median = abs(sample_median - mean_median) > 3 * sd_median,
    outlier_IQR    = abs(sample_IQR    - mean_IQR)    > 3 * sd_IQR,
    is_outlier     = outlier_median | outlier_IQR
  ) |>
  ungroup()

outlier_samples <- qc_stats |>
  filter(is_outlier) |>
  select(SampleID, Panel, outlier_median, outlier_IQR)

cat("Statistical outliers detected:", n_distinct(outlier_samples$SampleID), "samples\n")
print(outlier_samples)

In [ ]:
# Combine statistical outliers with Olink-flagged samples
samples_to_exclude <- union(
  outlier_samples$SampleID,
  qc_flagged$SampleID
)

cat("Total samples to exclude:", length(samples_to_exclude), "\n")
cat("Sample IDs:", paste(samples_to_exclude, collapse = ", "), "\n")

In [ ]:
# Remove outliers and re-run PCA to confirm improvement
npx_clean <- npx_samples |>
  filter(!SampleID %in% samples_to_exclude)

cat("Samples remaining after exclusion:",
    n_distinct(npx_clean$SampleID), "\n")

# PCA post-exclusion
pca_clean <- olink_pca_plot(npx_clean, label_samples = TRUE, quiet = TRUE)
pca_clean[[1]]

In [ ]:
ggsave("PCA_after_QC_exclusion.png", plot = pca_clean[[1]], width = 9, height = 7, dpi = 300)
message("Post-exclusion PCA saved.")

---
## Step 10 — Missing Frequency & Below-LOD Summary

For each assay, we calculate the percentage of samples with NPX below the Limit of Detection.

### Why this matters
- Proteins with high below-LOD rates have **poor detectability** in your sample type and may not be informative.
- However, Olink recommends **retaining** below-LOD data in statistical analyses — values below LOD still converge across groups and **do not inflate false positive rates**. Exclusion is most appropriate for technical evaluation (e.g. CV calculation) rather than statistical testing.
- A protein well-expressed in one group but undetected in another can be a **strong biomarker candidate**.

> A common (but flexible) exclusion threshold is **> 70% below LOD** across all samples. Adjust this based on your study design.

In [ ]:
lod_summary <- npx_clean |>
  group_by(OlinkID, Assay, Panel) |>
  summarise(
    pct_below_LOD     = mean(NPX < LOD, na.rm = TRUE) * 100,
    mean_MissingFreq  = mean(MissingFreq, na.rm = TRUE),
    median_NPX        = median(NPX, na.rm = TRUE),
    .groups           = "drop"
  ) |>
  arrange(desc(pct_below_LOD))

cat("Top 20 assays by % below LOD:\n")
print(head(lod_summary, 20))

In [ ]:
# Visualise the distribution of below-LOD rates across all assays
ggplot(lod_summary, aes(x = pct_below_LOD)) +
  geom_histogram(binwidth = 5, fill = "steelblue", colour = "white") +
  geom_vline(xintercept = 70, linetype = "dashed", colour = "red", linewidth = 0.8) +
  annotate("text", x = 72, y = Inf, label = "70% threshold",
           hjust = 0, vjust = 1.5, colour = "red", size = 3.5) +
  labs(
    title = "Distribution of % Samples Below LOD per Assay",
    x     = "% Samples Below LOD",
    y     = "Number of Assays"
  ) +
  theme_bw()

In [ ]:
# Set your exclusion threshold (adjust as needed)
lod_threshold <- 70

low_detect_assays <- lod_summary |>
  filter(pct_below_LOD > lod_threshold) |>
  pull(OlinkID)

cat("Assays with >", lod_threshold, "% below LOD:",
    length(low_detect_assays), "of", n_distinct(lod_summary$OlinkID), "total\n")

# Optionally remove low-detectability assays for downstream statistical analysis
npx_clean_filtered <- npx_clean |>
  filter(!OlinkID %in% low_detect_assays)

cat("Assays remaining after LOD filter:",
    n_distinct(npx_clean_filtered$OlinkID), "\n")

---
## Step 11 — Plate / Batch Effect Check

If your dataset spans **multiple plates or sequencing runs**, it is important to check for batch effects before downstream analysis.

### Signs of a batch effect
- Samples cluster by `PlateID` rather than by biological group in the PCA/UMAP
- The QC plot shows systematic shifts in IQR or median per plate

### Remediation
If a batch effect is detected, apply **bridging normalisation** using `olink_normalization()`. This requires bridge samples (ideally 8–16) run across all batches. See the `Introduction_to_bridging_Olink_NPX_datasets` vignette for full details.

In [ ]:
# UMAP coloured by PlateID — alternative to PCA for batch effect visualisation
# Uncomment if umap package is installed
# umap_plate <- olink_umap_plot(npx_clean, color_g = "PlateID", quiet = TRUE)
# umap_plate[[1]]

# Box plot of median NPX per plate — a direct numerical batch check
npx_clean |>
  group_by(SampleID, PlateID, Panel) |>
  summarise(sample_median = median(NPX, na.rm = TRUE), .groups = "drop") |>
  ggplot(aes(x = PlateID, y = sample_median, fill = PlateID)) +
  geom_boxplot(outlier.size = 0.8, alpha = 0.7) +
  facet_wrap(~Panel, scales = "free_y") +
  labs(
    title = "Sample Median NPX by Plate — Batch Effect Check",
    x     = "Plate ID",
    y     = "Median NPX"
  ) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

> **Interpretation:** If median NPX is systematically higher or lower on specific plates, bridging normalisation is recommended before statistical testing.

---
## Step 12 — Export QC-Passed Data

Export the following outputs:

| File | Content |
|------|---------|
| `Olink_Reveal_QC_passed.csv` | Clean NPX data ready for downstream analysis |
| `QC_outlier_samples.csv` | List of all excluded samples with reason |
| `LOD_summary_per_assay.csv` | Per-assay detectability summary |
| `QC_session_summary.txt` | Key numbers from this QC run |

In [ ]:
# Export clean dataset
write.csv(npx_clean_filtered,
          file      = "Olink_Reveal_QC_passed.csv",
          row.names = FALSE)

# Export outlier sample list
write.csv(
  data.frame(SampleID = samples_to_exclude,
             Source   = ifelse(samples_to_exclude %in% qc_flagged$SampleID,
                               "Olink_QC_flag", "Statistical_outlier")),
  file      = "QC_outlier_samples.csv",
  row.names = FALSE
)

# Export LOD summary
write.csv(lod_summary,
          file      = "LOD_summary_per_assay.csv",
          row.names = FALSE)

# Print a session summary
summary_text <- paste0(
  "=== Olink Reveal QC Session Summary ===",                          "\n",
  "Date: ",                Sys.time(),                                 "\n",
  "Original samples:      ", n_distinct(npx_samples$SampleID),       "\n",
  "Excluded samples:      ", length(samples_to_exclude),              "\n",
  "  - Olink QC flags:   ", nrow(qc_flagged),                        "\n",
  "  - Statistical (IQR/median outlier): ",
                             n_distinct(outlier_samples$SampleID),    "\n",
  "Remaining samples:     ", n_distinct(npx_clean$SampleID),         "\n",
  "Original assays:       ", n_distinct(npx_samples$OlinkID),        "\n",
  "Low-detectability assays removed (>",
                             lod_threshold, "% below LOD): ",
                             length(low_detect_assays),               "\n",
  "Final assays:          ", n_distinct(npx_clean_filtered$OlinkID), "\n"
)

cat(summary_text)
writeLines(summary_text, "QC_session_summary.txt")
message("\n✅ All QC outputs exported successfully.")

---
## 📋 Quick Reference

### Key OlinkAnalyze functions used in this workflow

| Function | Purpose |
|----------|---------|
| `read_NPX()` | Import NPX parquet/xlsx/csv into long format |
| `olink_lod()` | Add LOD to Explore/Reveal NGS datasets |
| `olink_qc_plot()` | IQR vs median scatter plot; flags ±3 SD outliers |
| `olink_pca_plot()` | PCA for global sample overview |
| `olink_dist_plot()` | NPX distribution density per panel |
| `olink_umap_plot()` | UMAP dimensionality reduction (alternative to PCA) |
| `olink_normalization()` | Bridge/subset/population/reference normalization |

### LOD method decision guide

```
Count passing NCs in your dataset
      │
      ├── ≥ 10 NCs  →  lod_method = "NCLOD"     (project-specific, preferred)
      │
      └──  < 10 NCs  →  lod_method = "FixedLOD"  (download CSV from olink.com)
```

### Next steps after QC

- **Batch correction** (if needed): `olink_normalization()` with bridge samples
- **Statistical testing**: `olink_ttest()`, `olink_wilcox()`, `olink_anova()`, `olink_lmer()`
- **Visualisation**: `olink_volcano_plot()`, `olink_heatmap_plot()`, `olink_boxplot()`
- **Pathway enrichment**: `olink_pathway_enrichment()` + `olink_pathway_visualization()`

---
*Olink® Analyze v3.x | OlinkAnalyze CRAN package | © 2025 Olink Proteomics AB, part of Thermo Fisher Scientific*